<a href="https://colab.research.google.com/github/Varad2001/Gender-classification-simple-project/blob/main/model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import keras 
from sklearn.model_selection import train_test_split 
import tensorflow as tf
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Dense, Dropout 
from keras.layers import Flatten, BatchNormalization
from tensorflow.keras.utils import to_categorical
import os
from PIL import Image
from PIL import UnidentifiedImageError

In [6]:
# load the model from the corresponding directory
!ls /content/drive/MyDrive/Colab_Notebooks/saved_model/my_model2
new_model = tf.keras.models.load_model("/content/drive/MyDrive/Colab_Notebooks/saved_model/my_model2")

assets	keras_metadata.pb  saved_model.pb  variables


In [7]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 50, 50, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 50, 50, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 25, 25, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 25, 25, 32)       128       
 hNormalization)                                                 
                                                        

Testing:<br>
1. Extract all images, convert them to nd-array and then add them to another nd-array.<br>
2. Reshape the resulting nd-array and get predictions from the model.<br>
3. Extract the probabilities for male or female label. <br>
4. e.g. If predicting male images, extract the first elements of the predictions, count how many are greater than 0.6-males, how many are less than 0.4- females and remaining - confused. <br>
5. Calculate the accuracy.

for every image, a list containing two elements is returned: <br>
first element : prob that the image belongs to male category<br>
second element : prob that the image belongs to female category<br>
e.g. <br>
[0.8, 0.2] means that the model thinks that there is 80% chance that the given image is male. <br>

Thus, we will extract the values of first elements for each image and compute the result as follows:<br>
if the value > 0.6 ==> the prediction is 'male'<br>
if the value < 0.4 ==> the prediction is 'female' <br>
else the model is confused

In [8]:
# define the function which converts imgs into arrays
def convert_img(img1):
    img = img1.convert("L")         # convert the image into greyscale
    px = 50
    img = img.resize((px,px))       # resize the image to 50*50 pixels, ==> total of 2500 pixels
    img = np.asarray(img)           # convert the image into numpy array, ==> shape will be (50,50)
    img = img / 255.0               # scale the pixel values between 0 to 1
    img = img.reshape(px,px,1)      # convert the two dimensional array to one-dimensional array
    return img

In [9]:
, UnidentifiedImageError
# function to predict the category and calculate accuracy
# takes input as file_path where image/s are stored ; returns dictionary containing results
# dict is returned in following format : {<img_file>:<category(fe/male)>}

def predict_gender(img_file_path):
  # store the filenames of each image in the img_file_path directory
  img_files = [img_file_path + '/' + img_file for img_file in os.listdir(img_file_path)]
  
  results = {}                   # store the results in the form of dict : {<file>:<male/female>}
  males , females  = 0,0
  
  i=0
  while i<len(img_files):        # for every image file : do following
      img_file = img_files[i]
      imgs = np.array([])
      
      try :
        img = Image.open(img_file)        # open the image object
      except Exception as e:              # the file is either not an image file or not supported
        img_files.remove(img_file)        # remove that file from the list
        continue
      
      img = convert_img(img)              # convert it to np-array
      imgs = np.append(imgs,img)          # store the array 
      imgs = imgs.reshape(1,50,50,1)      # input to the model must be provided as an array of shape : (no_of_imgs, px, px, 1)
      
      prediction = new_model.predict(imgs)
      male_prob = prediction[0][0]        # extract first element of prediction array which represents the prob of image being 'male' 
      if male_prob > 0.5:
        males += 1
        results[img_file]="male"          # store that file with the label
        print("Image category:Male")
      else :
        females += 1
        results[img_file]="female"
        print("Image category:Female")
      
      i+=1

  print("Male images:{} \nFemales images:{}".format(males,females))
  return results
  
  

In [10]:
# specify the path where the image/s is/are located
img_file_path = '/content/imgs'

In [11]:
results  = predict_gender(img_file_path)
results

Image category:Female
Image category:Female
Image category:Male
Image category:Male
Image category:Male
Image category:Male
Image category:Female
Image category:Male
Image category:Female
Male images:5 
Females images:4


{'/content/imgs/360_F_258899001_68CalsKTRk6PZQgWH9JhR4heBlncCko9.jpg': 'male',
 '/content/imgs/Chris-Evans-buzz-cut-e1538974047133.jpg': 'male',
 '/content/imgs/Rob-cc2d68e18be04acf90a74623c1087fd6.jpg': 'male',
 '/content/imgs/Stokes-Ageas-Squad.png': 'male',
 '/content/imgs/archetypal-female-_3249633c.jpg': 'female',
 '/content/imgs/ccd925bb9c3c0c5446f2b55c5597ee8c--nice-makeup-stunning-makeup.jpg': 'female',
 '/content/imgs/cook1.jpg': 'male',
 '/content/imgs/female-faces1.jpg': 'female',
 '/content/imgs/woman-6328478_960_720.jpg': 'female'}